In [15]:
#keras
import tensorflow.contrib.keras
#モデル作成用
from keras.models import Model
#レイヤー各種
from keras.layers import Dense, GlobalAveragePooling2D,Input
#既存学習済みモデル
from keras.applications.inception_v3 import InceptionV3

In [13]:
img_size = 224
ch = 3

In [14]:
base_model = InceptionV3(include_top = False,weights = "imagenet",input_tensor=Input(shape=(img_size,img_size,ch)))